<a href="https://colab.research.google.com/github/Kvazzzzar/MPSI/blob/main/MPSI_3.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import numpy as np
import random

class SimpleGPT:
    def __init__(self, vocab_size, embedding_dim=64, max_seq_length=20):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.max_seq_length = max_seq_length

        # Инициализация с более подходящими масштабами
        scale = 0.01
        self.token_emb = np.random.randn(vocab_size, embedding_dim) * scale
        self.pos_emb = np.random.randn(max_seq_length, embedding_dim) * scale
        self.Wq = np.random.randn(embedding_dim, embedding_dim) * scale
        self.Wk = np.random.randn(embedding_dim, embedding_dim) * scale
        self.Wv = np.random.randn(embedding_dim, embedding_dim) * scale
        self.Wo = np.random.randn(embedding_dim, vocab_size) * scale

        # Кэш для вычислений
        self.cache = {}

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

    def attention(self, x):
        Q = np.dot(x, self.Wq)
        K = np.dot(x, self.Wk)
        V = np.dot(x, self.Wv)

        scores = np.dot(Q, K.T) / np.sqrt(self.embedding_dim)
        weights = self.softmax(scores)
        output = np.dot(weights, V)

        # Сохраняем для обратного прохода
        self.cache['x'] = x
        self.cache['weights'] = weights
        self.cache['V'] = V

        return output

    def forward(self, token_ids):
        token_emb = self.token_emb[token_ids]
        pos_ids = np.arange(len(token_ids))
        pos_emb = self.pos_emb[pos_ids]

        x = token_emb + pos_emb
        x = self.attention(x)
        logits = np.dot(x, self.Wo)

        self.cache['token_ids'] = token_ids
        self.cache['pos_ids'] = pos_ids
        self.cache['x_attn'] = x

        return logits

    def compute_loss(self, logits, targets):
        probs = self.softmax(logits)
        loss = -np.mean(np.log(probs[np.arange(len(targets)), targets] + 1e-8))
        return loss, probs

    def backward(self, dlogits, learning_rate):
        # Градиенты выходного слоя
        x_attn = self.cache['x_attn']
        dWo = np.dot(x_attn.T, dlogits)

        # Градиенты через attention
        dx_attn = np.dot(dlogits, self.Wo.T)

        # Градиенты механизма внимания
        weights = self.cache['weights']
        V = self.cache['V']
        x = self.cache['x']

        dV = np.dot(weights.T, dx_attn)
        dweights = np.dot(dx_attn, V.T)

        # Градиенты Q, K, V
        Q = np.dot(x, self.Wq)
        K = np.dot(x, self.Wk)

        dQ = np.dot(dweights * (1 - weights), K) / np.sqrt(self.embedding_dim)
        dK = np.dot((dweights * (1 - weights)).T, Q) / np.sqrt(self.embedding_dim)
        dV = np.dot(weights.T, dx_attn)

        dWq = np.dot(x.T, dQ)
        dWk = np.dot(x.T, dK)
        dWv = np.dot(x.T, dV)

        # Обновление параметров
        self.Wo -= learning_rate * dWo
        self.Wq -= learning_rate * dWq
        self.Wk -= learning_rate * dWk
        self.Wv -= learning_rate * dWv

    def train(self, data, epochs=100, batch_size=8, learning_rate=0.01):
        losses = []
        for epoch in range(epochs):
            # Перемешиваем данные
            np.random.shuffle(data)
            total_loss = 0
            count = 0

            for i in range(0, len(data)-batch_size-1, batch_size):
                batch = data[i:i+batch_size]
                inputs, targets = batch[:-1], batch[1:]

                # Прямой проход
                logits = self.forward(inputs)
                loss, probs = self.compute_loss(logits, targets)
                total_loss += loss
                count += 1

                # Обратный проход
                dlogits = probs
                dlogits[np.arange(len(targets)), targets] -= 1
                dlogits /= len(targets)

                self.backward(dlogits, learning_rate)

            avg_loss = total_loss / count
            losses.append(avg_loss)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

            # Уменьшаем learning rate
            learning_rate *= 0.995

        return losses

    def generate(self, start_token, max_length=10, temperature=0.7):
        tokens = [start_token]
        for _ in range(max_length):
            logits = self.forward(np.array(tokens[-self.max_seq_length:]))
            probs = self.softmax(logits[-1]/temperature)
            next_token = np.random.choice(len(probs), p=probs)
            tokens.append(next_token)
        return tokens

# Пример использования с более крупным текстом
text_data = """
я вас любил любовь еще быть может
в душе моей угасла не совсем
но пусть она вас больше не тревожит
я не хочу печалить вас ничем
я вас любил безмолвно безнадежно
то робостью то ревностью томим
я вас любил так искренно так нежно
как дай вам бог любимой быть другим
""" * 10  # Повторяем текст для увеличения объема данных

# Предварительная обработка
text_data = re.sub(r'[^\w\s]', '', text_data.lower())
words = list(set(text_data.split()))
vocab = {word: i for i, word in enumerate(words)}
id_to_word = {i: word for word, i in vocab.items()}
train_data = [vocab[word] for word in text_data.split() if word in vocab]

print(f"Размер словаря: {len(vocab)}")
print(f"Длина обучающих данных: {len(train_data)}")

# Создаем и обучаем модель
gpt = SimpleGPT(vocab_size=len(vocab), embedding_dim=64, max_seq_length=16)
losses = gpt.train(np.array(train_data), epochs=100, batch_size=16, learning_rate=0.01)

# Генерация текста
print("\nГенерация текста:")
start_token = vocab.get("я", 0)
generated_tokens = gpt.generate(start_token, max_length=20, temperature=0.7)
generated_text = ' '.join([id_to_word.get(tok, '') for tok in generated_tokens])
print(f"Сгенерированный текст: {generated_text}")

Размер словаря: 36
Длина обучающих данных: 500
Epoch 1/100, Loss: 3.5835
Epoch 2/100, Loss: 3.5835
Epoch 3/100, Loss: 3.5835
Epoch 4/100, Loss: 3.5835
Epoch 5/100, Loss: 3.5835
Epoch 6/100, Loss: 3.5835
Epoch 7/100, Loss: 3.5835
Epoch 8/100, Loss: 3.5835
Epoch 9/100, Loss: 3.5835
Epoch 10/100, Loss: 3.5835
Epoch 11/100, Loss: 3.5835
Epoch 12/100, Loss: 3.5835
Epoch 13/100, Loss: 3.5835
Epoch 14/100, Loss: 3.5835
Epoch 15/100, Loss: 3.5835
Epoch 16/100, Loss: 3.5835
Epoch 17/100, Loss: 3.5835
Epoch 18/100, Loss: 3.5835
Epoch 19/100, Loss: 3.5835
Epoch 20/100, Loss: 3.5835
Epoch 21/100, Loss: 3.5835
Epoch 22/100, Loss: 3.5835
Epoch 23/100, Loss: 3.5835
Epoch 24/100, Loss: 3.5835
Epoch 25/100, Loss: 3.5835
Epoch 26/100, Loss: 3.5835
Epoch 27/100, Loss: 3.5835
Epoch 28/100, Loss: 3.5835
Epoch 29/100, Loss: 3.5835
Epoch 30/100, Loss: 3.5835
Epoch 31/100, Loss: 3.5835
Epoch 32/100, Loss: 3.5835
Epoch 33/100, Loss: 3.5835
Epoch 34/100, Loss: 3.5835
Epoch 35/100, Loss: 3.5835
Epoch 36/100, Los